In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import tikzplotlib

from src.config import SAT_TEST_SEQ, POLICIES

COLOURS = ["blue", "orange", "green", "red"]
ENCODING_DIR = "./results/encoding"
POLICY_DIR = "./results/policy"
SOLVER_DIR = "./results/solver"
TEX_DIR = "./tex"

In [ ]:
# Toggle what results to show
SHOW_ENCODING_RESULTS = True
SHOW_SAT_RESULTS = True
SHOW_SOLVER_RESULTS = True

In [ ]:
# Get a list of the results from a directory
def get_result_dicts(dir_path: str) -> list[dict[str, float | str]]:
    results = []
    for file_name in filter(lambda x: x.endswith(".csv"), os.listdir(dir_path)):
        # Open the csv file
        file_dict = pd.read_csv(os.path.join(dir_path, file_name)).to_dict()
        # For each header in the csv where (key1=name,val1=dict(key=row_num,val=val_at_row))
        valid = True
        for key1, val1 in file_dict.items():
            # If the value is a string, convert from a dict of strings to 1 string
            if len(val1) == 0:
                valid = False
                break
            if type(val1[0]) == str:
                file_dict[key1] = val1[0]
            # Else find the average of the values
            else:
                values = [v for v in val1.values()]
                file_dict[key1] = sum(values) / len(values)
        if valid:
            results.append(file_dict)
    return results

# Helper functions for working with return value of the above function
def get_all_vars_for(result_dicts: list[dict[str, float | str]], var_name: str) -> list:
    return sorted(set(x[var_name] for x in result_dicts))

def filter_by(result_dicts: list[dict[str, float | str]], var_name: str, var: str | float) -> list:
    return [x for x in result_dicts if x[var_name] == var]

# Given the results, plot the change in time per length for the independent variable
def plot_data(result_dicts: list[dict[str, float | str]], indp_var: str):
    # TODO: Complete function to plot data to compare indp vars, where x axis is length, y axis is time taken
    mapped_results: dict[str, list[dict]] = {}
    fig, ax_left = plt.subplots(figsize=(16, 10))
    ax_right = ax_left.twinx()

    for r in result_dicts:
        key = r[indp_var]
        mapped_results[key] = mapped_results.get(key, []) + [r]

    # Loop through each 
    for i, (var, results) in enumerate(mapped_results.items()):
        data = []
        incl = ["len", "encode_time", "total_time", "sat_time"]
        for result in results:
            data.append({i: result[i] for i in incl})
        data.sort(key=lambda x:x["len"])


# Encoding Test Results

The following code compares for each policy the best combination of `[search-policy][sat-solver][encoding-version]`

In [ ]:
if SHOW_ENCODING_RESULTS:
    #  Compare the performance of each (policy, solver, version) combination
    results = get_result_dicts(POLICY_DIR)
    policies = get_all_vars_for(results, "policy")
    solvers = get_all_vars_for(results, "solver")
    vers = get_all_vars_for(results, "ver")
    lengths = get_all_vars_for(results, "len")[:-1] # Ignore last length since not everything was tested
    print(lengths)

    # Results
    results_each_len: dict[str, dict[str, dict[list[float]]]] = {
        policy: {
            solver: {
                ver: [] for ver in vers
            } for solver in solvers
        } for policy in policies
    }

    print(" " * (1 + len(policies[0]) + len(solvers[0])), "   \t", "[v0,      v1,     v2]")
    policy_times_per_len = {
        policy: {
            length: [] for length in lengths
        } for policy in policies
    }

    # Get the times for all policies
    for policy in policies:
        policy_times = []
        # Get the times for all solvers
        for solver in solvers:
            policy_solver_results = filter_by(filter_by(results, "policy", policy), "solver", solver)
            ver_results = []
            # Get the time for all versions
            for ver in vers:
                policy_solver_ver_results = filter_by(policy_solver_results, "ver", ver)
                avg_ver_results = np.mean([x["total_time"] for x in policy_solver_ver_results])
                ver_results.append(round(avg_ver_results, 4))

                # Get the time for all lengths
                for length in lengths:
                    length_results = filter_by(policy_solver_ver_results, "len", length)
                    results_each_len[policy][solver][ver].append(np.mean([x["total_time"] for x in length_results]))
                    avg_times = np.mean([x["total_time"] for x in length_results])
                    policy_times_per_len[policy][length].append(avg_times)
            
            # Print out the times for current policy and solver
            ver_avg = round(np.mean(ver_results), 4)
            policy_times.append(ver_avg)
            print(policy, solver, "   \t", ver_results, f"\t{ver_avg = }")
        
        # Print out the average time for the policy
        policy_avg = round(np.mean(policy_times), 4)
        print(f"{policy_avg = }")
        print()

    # Plot the times taken
    plt.figure(figsize=(14, 10))
    for p in policies:
        # TODO: Show average for all solvers and all encodings
        times = []

        for l in lengths:
            times.append(np.mean(policy_times_per_len[p][l]))
        
        plt.plot(lengths, times, label=p.replace("_", " "))
    plt.legend()
    scale_mode = "linear"
    plt.yscale(scale_mode)
    tikzplotlib.save(os.path.join(TEX_DIR, f"policy-{scale_mode}.tex"))
    plt.show()

# SAT Test Results

The following code shows how long it takes for a kissat to solve for different goal contacts.

In [ ]:
if SHOW_SAT_RESULTS:
    with open(SAT_TEST_SEQ.replace("input", "results/sat") + ".json") as f:
        data = json.load(f)
        plt.figure(figsize=(14, 10))
        times = data["times"]
        n = len(times)
        calls = [i + 1 for i in range(n)]
        plt.bar(calls, times)
        # plt.yscale("log")
        tikzplotlib.save(os.path.join(TEX_DIR, "sat.tex"))
        plt.show()
        print(f"{times = }")
        for policy in POLICIES:
            calls = data[policy]
            total = sum([times[i - 1] for i in calls])
            print(policy, total, len(calls), calls)

# Solver Test Results

The following code compares how long it takes to solve for different combinations of solvers and encodings

In [ ]:
if SHOW_SOLVER_RESULTS:
    #  Compare the performance of each (solver, version) combination
    results = get_result_dicts(SOLVER_DIR)
    solvers = get_all_vars_for(results, "solver")
    vers = get_all_vars_for(results, "ver")
    lengths = get_all_vars_for(results, "len")[:-1]

    # Results
    results_each_len: dict[str, dict[str, dict[list[float]]]] = {
        solver: {
            ver: [] for ver in vers
        } for solver in solvers
    }

    solver_times_per_len = {
        solver: {
            length: [] for length in lengths
        } for solver in solvers
    }

    print(" " * (1 + len(solvers[0])), "   \t", "[v0,      v1,     v2]")
    # Get the times for all solvers
    for solver in solvers:
        solver_results = filter_by(results, "solver", solver)
        ver_results = []
        # Get the time for all versions
        for ver in vers:
            solver_ver_results = filter_by(solver_results, "ver", ver)
            avg_ver_results = np.mean(
                [x["total_time"] for x in solver_ver_results])
            ver_results.append(round(avg_ver_results, 4))

            # Get the time for all lengths
            for length in lengths:
                length_results = filter_by(
                    solver_ver_results, "len", length)
                avg_times = np.mean([x["total_time"] for x in length_results])
                results_each_len[solver][ver].append(avg_times)
                solver_times_per_len[solver][length].append(avg_times)

        # Print out the times for current policy and solver
        ver_avg = round(np.mean(ver_results), 4)
        print(solver, "   \t", ver_results, f"\t{ver_avg = }")

    # Plot the times taken
    plt.figure(figsize=(14, 10))
    for s in solvers:
        times = []
        for l in lengths:
            times.append(np.mean(solver_times_per_len[s][l]))
        plt.plot(lengths, times, label=s)
    plt.legend()
    scale_mode = "linear"
    plt.yscale(scale_mode)
    tikzplotlib.save(os.path.join(TEX_DIR, f"solver-{scale_mode}.tex"))
    plt.show()

# Encoding Test Results

The following code compares how long it takes to solve for the different encoding versions to solve 3D instances

In [ ]:
dimension = 3
yscale_mode = "linear"
results = filter_by(get_result_dicts(ENCODING_DIR), "dim", dimension)
vers = get_all_vars_for(results, "ver")
lengths = get_all_vars_for(results, "len")[:-1]
dims = get_all_vars_for(results, "dim")

fig, ax_left = plt.subplots(figsize=(16, 10))
ax_right = ax_left.twinx()
for i, ver in enumerate(vers):
    ver_results = filter_by(results, "ver", ver)
    length_times = []
    length_cls = []
    length_vars = []
    for length in lengths:
        ver_len_result = filter_by(ver_results, "len", length)
        times = [x["total_time"] for x in ver_len_result]
        cls = [x["cls"] for x in ver_len_result]
        vars = [x["vars"] for x in ver_len_result]
        length_times.append(np.mean(times))
        length_cls.append(np.mean(cls))
        length_vars.append(np.mean(vars))
    ax_left.plot(lengths, length_times, color=COLOURS[i], marker="o", label=f"{ver} times")
    ax_right.plot(lengths, length_cls, ":", color=COLOURS[i], label=f"{ver} clauses")
    ax_right.plot(lengths, length_vars, "--", color=COLOURS[i], label=f"{ver} variables")

ax_left.set_yscale(yscale_mode)
ax_left.set_ylabel("Time (s)")

ax_right.set_yscale(yscale_mode)
ax_right.set_ylabel("Variables / Clauses")
# ax_left.grid(True, which="minor")
ax_left.set_xlabel("Sequence Length")

ax_left.legend()
ax_right.legend()
plt.show()